# AGENDA

### 1. Possible Predictors - kaggle data
### 2. Economist Conversations and Problem Understanding - Shantanu, John
### 3. Data Collection Strategy - market + News (multipe sources)

- https://www.cnbc.com/soybeans/
- https://www.agweb.com/crops/soybeans
- http://sfntoday.com/?s=soybean
- https://markets.businessinsider.com/commodities/news/soybeans-price
- https://www.foodbusinessnews.net/search?q=soybean

Supply and Demand Monthly US and World: https://www.usda.gov/oce/commodity/wasde/

### 4. Correlation towards predicting prices
### 5. Findings Sharing

### Shantanu , Himanshu- Data Collection, Exploration, Elimination

Initial Goal is to answer below starter Questions:

#### Answer Undergrad Questions:
    1. What is the price difference by contract date?
    2. Is there seasonality in the target variable for a given contract date?
    3. Is canola price predictive of soybean price?
    4. Is there an effect of the delayed 2019 seeding on soybean prices (see time lag in data sets on Oil Crops Outlook tab: oil crops chart gallery figure 1)?
#### Answer Grad questions:
    5. Is there a lag between tweets related to agriculture and/or trade and the markets?
    6. What are the most impact indicators to watch from the next top three soybean producing
    countries (Brazil, Argentina and China)?
    7. What other data sources could you use as macroeconomic indicators, and what is their relative
    importance?

## Table of Content
    1. Understanding the Data
    2. Data Exploration
    3. Summary

## Step 1: Understanding the Data

- Load Data and Fix issues if any
- Sample Rows: 454 and 5 features for each contract March2020(ZSH2020), May2020(ZSK2020) and July2020(ZSN2020). (target - Close price)
- Except Date, All numerical Columns
- Date is common across all three contract dates
- Removed last two **NaN** rows, No duplicates

In [1]:
import re
import pdb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

In [2]:
#Supporting Functions
def rename_col(df, suffix):
    for col_name in list(df):
        col_name_new = col_name +  "_" + suffix
        df.rename(columns={col_name: col_name_new}, inplace=True)
        
    return df

In [3]:
ug_data_mar = pd.read_csv('./Data_Undergrad/ActiveSoybeanContractsForMarch2020.csv')
ug_data_mar['Date'] = ug_data_mar['Date'].astype('datetime64[ns]')
ug_data_mar = rename_col(ug_data_mar, "mar")
print(f'{ug_data_mar.shape[0]} samples and {ug_data_mar.shape[1]} features in the March 2020 contract dataset.')
ug_data_mar.head()

454 samples and 5 features in the March 2020 contract dataset.


,Date_mar,Open_mar,High_mar,Low_mar,Close_mar
0,2017-11-14,978.5,978.5,978.5,978.5
1,2017-11-15,985.0,985.0,985.0,985.0
2,2017-11-16,983.0,983.0,983.0,983.0
3,2017-11-17,996.5,996.5,996.5,996.5
4,2017-11-20,998.5,998.5,998.5,998.5


In [4]:
ug_data_may = pd.read_csv('./Data_Undergrad/ActiveSoybeanContractsForMay2020.csv')
ug_data_may['Date'] = ug_data_may['Date'].astype('datetime64[ns]')
ug_data_may = rename_col(ug_data_may, "may")
print(f'{ug_data_may.shape[0]} samples and {ug_data_may.shape[1]} features in the May 2020 contract  dataset.')
ug_data_may.tail()

454 samples and 5 features in the May 2020 contract  dataset.


,Date_may,Open_may,High_may,Low_may,Close_may
449,2019-08-28,897.25,904.75,890.75,902.75
450,2019-08-29,902.75,907.75,902.50,904.75
451,2019-08-30,904.00,913.25,903.00,904.50
452,NaT,NaN,NaN,NaN,NaN
453,NaT,NaN,NaN,NaN,NaN


In [5]:
ug_data_july = pd.read_csv('./Data_Undergrad/ActiveSoybeanContractsForJuly2020.csv')
ug_data_july['Date'] = ug_data_july['Date'].astype('datetime64[ns]')
ug_data_july = rename_col(ug_data_july, "july")
print(f'{ug_data_july.shape[0]} samples and {ug_data_july.shape[1]} features in the July 2020 contract dataset.')
ug_data_july.head()

706 samples and 5 features in the July 2020 contract dataset.


,Date_july,Open_july,High_july,Low_july,Close_july
0,2016-11-14,947.00,947.00,947.00,947.00
1,2016-11-15,945.50,945.50,945.50,945.50
2,2016-11-16,946.00,946.00,946.00,946.00
3,2016-11-17,951.75,951.75,951.75,951.75
4,2016-11-18,944.75,944.75,944.75,944.75


In [6]:
ug_data_july.tail()

,Date_july,Open_july,High_july,Low_july,Close_july
701,2019-08-28,908.50,915.00,901.25,913.00
702,2019-08-29,913.00,918.00,912.75,914.75
703,2019-08-30,914.75,923.25,913.00,914.25
704,NaT,NaN,NaN,NaN,NaN
705,NaT,NaN,NaN,NaN,NaN


In [7]:
ug_data_july.head()

,Date_july,Open_july,High_july,Low_july,Close_july
0,2016-11-14,947.00,947.00,947.00,947.00
1,2016-11-15,945.50,945.50,945.50,945.50
2,2016-11-16,946.00,946.00,946.00,946.00
3,2016-11-17,951.75,951.75,951.75,951.75
4,2016-11-18,944.75,944.75,944.75,944.75


In [8]:
#Concating March, May and July prices 
result = pd.concat([ug_data_mar, ug_data_may], axis=1)
result = pd.merge(ug_data_july, result,  left_on='Date_july', right_on='Date_mar', how='outer')
#pd.concat([result, ug_data_july], axis=1)
#result.drop(['Date_mar', 'Date_may'], axis=1, inplace=True)
result.shape

(708, 15)

In [9]:
result.tail()

,Date_july,Open_july,High_july,Low_july,Close_july,Date_mar,Open_mar,High_mar,Low_mar,Close_mar,Date_may,Open_may,High_may,Low_may,Close_may
703,2019-08-30,914.75,923.25,913.0,914.25,2019-08-30,893.5,903.25,892.25,894.25,2019-08-30,904.0,913.25,903.0,904.5
704,NaT,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
705,NaT,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
706,NaT,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
707,NaT,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN


In [10]:
result = result.drop(704)
result = result.drop(705)
result = result.drop(706)
result = result.drop(707)
result.tail()

,Date_july,Open_july,High_july,Low_july,Close_july,Date_mar,Open_mar,High_mar,Low_mar,Close_mar,Date_may,Open_may,High_may,Low_may,Close_may
699,2019-08-26,908.25,919.75,907.50,914.75,2019-08-26,883.50,897.75,883.50,893.50,2019-08-26,895.75,909.00,895.75,904.25
700,2019-08-27,913.00,913.50,906.00,908.00,2019-08-27,891.50,892.00,883.25,886.25,2019-08-27,903.00,903.00,894.75,897.25
701,2019-08-28,908.50,915.00,901.25,913.00,2019-08-28,886.25,894.00,879.50,892.00,2019-08-28,897.25,904.75,890.75,902.75
702,2019-08-29,913.00,918.00,912.75,914.75,2019-08-29,891.50,897.50,891.50,894.00,2019-08-29,902.75,907.75,902.50,904.75
703,2019-08-30,914.75,923.25,913.00,914.25,2019-08-30,893.50,903.25,892.25,894.25,2019-08-30,904.00,913.25,903.00,904.50


In [11]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 704 entries, 0 to 703
Data columns (total 15 columns):
Date_july     704 non-null datetime64[ns]
Open_july     704 non-null float64
High_july     704 non-null float64
Low_july      704 non-null float64
Close_july    704 non-null float64
Date_mar      452 non-null datetime64[ns]
Open_mar      452 non-null float64
High_mar      452 non-null float64
Low_mar       452 non-null float64
Close_mar     452 non-null float64
Date_may      452 non-null datetime64[ns]
Open_may      452 non-null float64
High_may      452 non-null float64
Low_may       452 non-null float64
Close_may     452 non-null float64
dtypes: datetime64[ns](3), float64(12)
memory usage: 88.0 KB


In [12]:
#NaN rows
result.isnull().sum()

Date_july       0
Open_july       0
High_july       0
Low_july        0
Close_july      0
Date_mar      252
Open_mar      252
High_mar      252
Low_mar       252
Close_mar     252
Date_may      252
Open_may      252
High_may      252
Low_may       252
Close_may     252
dtype: int64

In [13]:
result.tail()

,Date_july,Open_july,High_july,Low_july,Close_july,Date_mar,Open_mar,High_mar,Low_mar,Close_mar,Date_may,Open_may,High_may,Low_may,Close_may
699,2019-08-26,908.25,919.75,907.50,914.75,2019-08-26,883.50,897.75,883.50,893.50,2019-08-26,895.75,909.00,895.75,904.25
700,2019-08-27,913.00,913.50,906.00,908.00,2019-08-27,891.50,892.00,883.25,886.25,2019-08-27,903.00,903.00,894.75,897.25
701,2019-08-28,908.50,915.00,901.25,913.00,2019-08-28,886.25,894.00,879.50,892.00,2019-08-28,897.25,904.75,890.75,902.75
702,2019-08-29,913.00,918.00,912.75,914.75,2019-08-29,891.50,897.50,891.50,894.00,2019-08-29,902.75,907.75,902.50,904.75
703,2019-08-30,914.75,923.25,913.00,914.25,2019-08-30,893.50,903.25,892.25,894.25,2019-08-30,904.00,913.25,903.00,904.50


In [14]:
#Remove duplicates if any
result.drop_duplicates(inplace=True)
print(f'{result.shape[0]} samples and {result.shape[1]} features in the concatenated contract dataset.')

704 samples and 15 features in the concatenated contract dataset.


#### Answer 1: What is the price difference by contract date?

**There is multiple instances where difference of july-may inversely proportional to difference of may-mar prices.**

| Price | Mar | May | July | Comments |
| --- | --- | --- |--- | --- |
|Open'18 | Always low | stays middle | Always high | Steep fall - All contracts between June 25 to July 27 18, Aug 21 to Oct 5 18.|
|Open'19 | Always low | stays middle | Always high| Steep fall - May 8 to May 28 2019, Period of May2019 and July2019 Closing|

**Note:**

**One instance Mar and May contract crosses july but july again plunges back in a day or two.**
**High, Low and Close prices follows almost same trend as Opening price.**


#### Answer 2: Is there seasonality in the target variable for a given contract date?

    Seasonality - Festivals, events , Any predictable fluctuation or pattern that recurs or repeats over a one-year period is said to be seasonal. 
    
#### Given Data:
             
        Contract March2020(ZSH2020) Nov 14,2017 to Aug 30,2019 - 3 Years
        Contract May2020(ZSK2020) Nov 14,2017 to Aug 30,2019 - 3 Years
        Contract July2020(ZSN2020) Nov 14,2016 to Aug 30,2019 - 4 Years
        
 #### 2019 Contract Duration and Timeline
        
| Contract | Start | End | Duration | Comments |
| --- | --- | --- |--- | --- |
| ZSH2019 | Nov 1, 2016 | Mar 11, 2019 | 2 yr 4 Months | -- |
| ZSK2019 | Nov 1, 2016 | May 13, 2019 | 2 yr 6 Months | -- |
| ZSN2019 | Nov 14, 2015 | July 8, 2019 | 3 yr 8 Months | Longest |

**Note:** There must be yearly contract overlaps on next maturing contract.

In [15]:
#Calculate open , low , high and close differences
result["open_may_mar"] = result["Open_may"]-result["Open_mar"]
result["open_july_may"] = result["Open_july"]-result["Open_may"]

In [30]:
result.head()

,Date_july,Open_july,High_july,Low_july,Close_july,Date_mar,Open_mar,High_mar,Low_mar,Close_mar,Date_may,Open_may,High_may,Low_may,Close_may,open_may_mar,open_july_may
0,2016-11-14,947.00,947.00,947.00,947.00,NaT,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
1,2016-11-15,945.50,945.50,945.50,945.50,NaT,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2,2016-11-16,946.00,946.00,946.00,946.00,NaT,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
3,2016-11-17,951.75,951.75,951.75,951.75,NaT,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
4,2016-11-18,944.75,944.75,944.75,944.75,NaT,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
#Generic price change plot line comparison
fig = go.Figure()

cols = list(result)
unwanted_col = {"Date_july", 'Date_mar', 'Date_may'} 
  
cols = [ele for ele in cols if ele not in unwanted_col] 

data = []
for col_name in cols:
    
    data.append(go.Scatter(x=result['Date_july'].dt.strftime(date_format='%Y-%m-%d'), y=result[col_name],
                    #mode='lines',
                    name=col_name))
    #pdb.set_trace()
layout = go.Layout(dict(title = "Contract Prices",
                  xaxis = dict(title = ''),
                  yaxis = dict(title = 'Price (USD)'),
                  ),legend=dict(
                orientation="h"))
py.iplot(dict(data=data, layout=layout), filename='basic-line', image_width=1200,
    image_height=800)

In [17]:
#General stats prices deviations
result[cols].describe()

,Open_july,High_july,Low_july,Close_july,Open_mar,High_mar,Low_mar,Close_mar,Open_may,High_may,Low_may,Close_may,open_may_mar,open_july_may
count,704.000000,704.000000,704.000000,704.000000,452.000000,452.000000,452.000000,452.000000,452.000000,452.000000,452.000000,452.000000,452.00000,452.000000
mean,969.263849,971.190696,967.351562,969.273438,953.485619,956.479535,950.163717,953.265487,960.133850,962.892699,957.193584,960.044248,6.64823,7.389381
std,30.679304,29.125045,32.120921,30.604806,37.308325,35.365666,39.358327,37.494017,36.213139,34.230985,38.090428,36.184460,3.95553,3.726767
min,877.750000,880.250000,866.000000,878.000000,855.500000,856.000000,841.500000,852.750000,865.000000,867.750000,854.000000,865.000000,-24.00000,-13.000000
25%,951.937500,953.187500,947.750000,950.937500,927.125000,930.250000,920.875000,926.125000,934.375000,937.250000,928.937500,934.437500,4.75000,5.750000
50%,966.625000,968.250000,965.750000,966.750000,952.875000,956.250000,948.625000,953.750000,959.875000,963.250000,955.500000,959.875000,7.50000,7.500000
75%,995.812500,996.062500,995.812500,996.000000,986.375000,986.375000,986.375000,986.375000,992.250000,992.250000,992.250000,992.250000,8.25000,9.250000
max,1025.000000,1025.000000,1024.750000,1024.750000,1014.250000,1014.250000,1014.250000,1014.250000,1021.750000,1021.750000,1021.750000,1021.750000,27.50000,33.500000


#### 4. Is there an effect of the delayed 2019 seeding on soybean prices ?

May, June and July are high volume seeding months and Contract rates are falling for those months.

**There is very less to no impact because of delayed 2019 seeding on contract soybean prices.**

In [18]:
import datetime
soybean_outlook_lag = pd.read_csv('./Data_Undergrad/soybean_outlook_lag.csv')
soybean_outlook_lag['Date'] = soybean_outlook_lag['Date'].astype('datetime64[ns]')
#soybean_outlook_lag['Date'] = soybean_outlook_lag['Date'] + datetime.timedelta(days=1) #Bcuz outlook date is on sunday

In [19]:
outlook_cols = list(soybean_outlook_lag)

In [20]:
#contract_prices = pd.merge(result, soybean_outlook_lag, left_on='Date_july', right_on='Date')
#contract_prices.drop(outlook_cols, axis=1, inplace=True)
contract_prices = result[result["Date_july"] >= "2019-04-22"]
contract_prices.reset_index(inplace=True)
contract_prices.shape

(93, 18)

In [21]:
contract_prices.head()

,index,Date_july,Open_july,High_july,Low_july,Close_july,Date_mar,Open_mar,High_mar,Low_mar,Close_mar,Date_may,Open_may,High_may,Low_may,Close_may,open_may_mar,open_july_may
0,611,2019-04-22,948.00,948.25,943.25,944.00,2019-04-22,930.50,932.75,927.00,927.50,2019-04-22,938.00,939.75,934.25,934.50,7.50,10.00
1,612,2019-04-23,945.25,945.25,931.25,932.75,2019-04-23,928.00,929.00,914.00,915.00,2019-04-23,936.00,936.25,921.75,923.00,8.00,9.25
2,613,2019-04-24,932.50,934.50,925.50,926.00,2019-04-24,915.00,917.25,907.50,907.75,2019-04-24,923.50,925.00,915.50,916.25,8.50,9.00
3,614,2019-04-25,929.00,932.50,924.50,929.75,2019-04-25,908.75,914.75,906.50,912.00,2019-04-25,916.50,923.00,914.75,920.00,7.75,12.50
4,615,2019-04-26,926.25,929.25,923.25,924.25,2019-04-26,911.50,911.75,905.25,906.25,2019-04-26,918.75,920.00,913.50,914.50,7.25,7.50


In [22]:
from plotly.subplots import make_subplots
#Generic price change plot line comparison
fig = go.Figure()

cols = list(soybean_outlook_lag)
unwanted_col = {"Date"} 
  
cols = [ele for ele in cols if ele not in unwanted_col] 

data = []
for col_name in cols:
    
    data.append(go.Scatter(x=soybean_outlook_lag['Date'].dt.strftime(date_format='%Y-%m-%d'), y=soybean_outlook_lag[col_name],
                    #mode='lines',
                    name=col_name))
    #pdb.set_trace()
layout = go.Layout(dict(title = "Usual Soybean Lifespan",
                  xaxis = dict(title = ''),
                  yaxis = dict(title = 'In making - size count'),
                  ),legend=dict(
                orientation="h"))

py.iplot(dict(data=data, layout=layout), filename='basic-line', image_width=1200,
    image_height=800)


cols = list(contract_prices)
unwanted_col = {"Date_july", "Date_mar", "Date_may", "index"} 
  
cols = [ele for ele in cols if ele not in unwanted_col] 

data = []
for col_name in cols:
    
    data.append(go.Scatter(x=contract_prices['Date_july'].dt.strftime(date_format='%Y-%m-%d'), y=contract_prices[col_name],
                    #mode='lines',
                    name=col_name))
    #pdb.set_trace()
layout = go.Layout(dict(title = "Contract Prices",
                  xaxis = dict(title = ''),
                  yaxis = dict(title = 'Prices (USD)'),
                  ),legend=dict(
                orientation="h"))

py.iplot(dict(data=data, layout=layout), filename='basic-line', image_width=1200,
    image_height=800)

#### 5. Is there a lag between tweets related to agriculture and/or trade and the markets?

In [23]:
soybean_tweet_lag = pd.read_csv('./Data_Graduate/China_tweets_realDonaldTrump.csv')
print(f'{soybean_tweet_lag.shape[0]} samples and {soybean_tweet_lag.shape[1]} features in the China Tweets dataset.')
#soybean_tweet_lag['created_at'] = soybean_tweet_lag['created_at'].astype('datetime64[ns]')
#soybean_tweet_lag['created_at'] = soybean_tweet_lag['created_at']
soybean_tweet_lag['created_at']  = pd.to_datetime(soybean_tweet_lag['created_at'], format='%m/%d/%Y %H:%M')
soybean_tweet_lag.info()

1336 samples and 7 features in the China Tweets dataset.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1336 entries, 0 to 1335
Data columns (total 7 columns):
source            1336 non-null object
text              1336 non-null object
created_at        1336 non-null datetime64[ns]
retweet_count     1336 non-null int64
favorite_count    1336 non-null int64
is_retweet        1336 non-null bool
id_str            1336 non-null float64
dtypes: bool(1), datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 64.0+ KB


In [24]:
#NaN rows
soybean_tweet_lag.isnull().sum()

source            0
text              0
created_at        0
retweet_count     0
favorite_count    0
is_retweet        0
id_str            0
dtype: int64

In [25]:
soybean_tweet_lag['created_at_date'] = pd.to_datetime([d.date() for d in soybean_tweet_lag['created_at']])
soybean_tweet_lag['created_at_time'] = [d.time() for d in soybean_tweet_lag['created_at']]
#soybean_tweet_lag['created_at_date']  = pd.to_datetime(soybean_tweet_lag['created_at_date'], format='%Y-%m-%d')
tweet_countby_date = pd.DataFrame({'china_count' : soybean_tweet_lag.groupby( [ "created_at_date"] ).size()}).reset_index()
tweet_countby_date['created_at_date']  = pd.to_datetime(tweet_countby_date['created_at_date'], format='%Y-%m-%d')
tweet_countby_date.head()

,created_at_date,china_count
0,2011-01-27,2
1,2011-07-18,2
2,2011-07-19,2
3,2011-07-20,2
4,2011-07-27,2


In [26]:
tweet_ct = tweet_countby_date[tweet_countby_date["created_at_date"] >= "2019-04-22"]
tweet_ct.reset_index(inplace=True)
cols = list(tweet_ct)
unwanted_col = {"created_at_date", "index"} 
  
cols = [ele for ele in cols if ele not in unwanted_col] 

data = []
for col_name in cols:
    
    data.append(go.Scatter(x=tweet_ct['created_at_date'].dt.strftime(date_format='%Y-%m-%d'), y=tweet_ct[col_name],
                    #mode='lines',
                    name=col_name))
    #pdb.set_trace()
layout = go.Layout(dict(title = "Tweets China",
                  xaxis = dict(title = ''),
                  yaxis = dict(title = 'Tweet Count'),
                  ),legend=dict(
                orientation="h"))

py.iplot(dict(data=data, layout=layout), filename='basic-line', image_width=1200,
    image_height=800)

In [27]:
farmer_tweet_lag = pd.read_csv('./Data_Graduate/FarmerTweets_realDonaldTrump.csv')
print(f'{farmer_tweet_lag.shape[0]} samples and {farmer_tweet_lag.shape[1]} features in the Farmer Tweets dataset.')
farmer_tweet_lag['created_at']  = pd.to_datetime(farmer_tweet_lag['created_at'], format='%m-%d-%Y %H:%M:%S')
farmer_tweet_lag['created_at_date'] = pd.to_datetime([d.date() for d in farmer_tweet_lag['created_at']])
farmer_tweet_lag['created_at_time'] = [d.time() for d in farmer_tweet_lag['created_at']]
farmer_tweet_lag.head()

112 samples and 7 features in the Farmer Tweets dataset.


,source,text,created_at,retweet_count,favorite_count,is_retweet,id_str,created_at_date,created_at_time
0,Twitter for iPhone,...Trade Agreement.” @business @ChuckGrassley ...,2019-09-03 02:37:54,10243,50333,False,1168714708691632129,2019-09-03,02:37:54
1,Twitter for iPhone,....Trade Agreement.” @business @ChuckGrassley...,2019-09-02 21:58:14,5905,22528,False,1168644326982901763,2019-09-02,21:58:14
2,Twitter for iPhone,Just watched Congresswoman Debbie Dingell and ...,2019-08-31 18:29:37,15666,67513,False,1167867052410802178,2019-08-31,18:29:37
3,Twitter for iPhone,RT @SenatorBraun: Representing Indiana on the ...,2019-08-31 11:54:05,2218,0,True,1167767510969737216,2019-08-31,11:54:05
4,Twitter for iPhone,The Farmers are going to be so happy when they...,2019-08-29 13:19:47,18364,80977,False,1167064301565358080,2019-08-29,13:19:47


In [28]:
farmertweet_countby_date = pd.DataFrame({'farmer_count' : farmer_tweet_lag.groupby( [ "created_at_date"] ).size()}).reset_index()
farmertweet_countby_date['created_at_date']  = pd.to_datetime(farmertweet_countby_date['created_at_date'], format='%Y-%m-%d')
farmertweet_countby_date.head()

,created_at_date,farmer_count
0,2013-01-11,1
1,2013-01-28,1
2,2016-09-23,1
3,2016-10-24,1
4,2017-03-21,1


In [29]:
tweets_count = pd.merge(tweet_countby_date, farmertweet_countby_date, how='outer')
tweets_count = pd.merge(tweets_count, soybean_countby_date, how='outer')

#pd.concat([result, ug_data_july], axis=1)
tweets_count.drop(['index'], axis=1, inplace=True)
tweets_count.shape

NameError: name 'soybean_countby_date' is not defined

In [ ]:
tweets_count.head()

In [ ]:
#NaN rows
farmer_tweet_lag.isnull().sum()

In [ ]:
tweet_ct = tweets_count[tweets_count["created_at_date"] >= "2019-04-22"]
tweet_ct.reset_index(inplace=True)
cols = list(tweet_ct)
unwanted_col = {"created_at_date", "index"} 
  
cols = [ele for ele in cols if ele not in unwanted_col] 

data = []
for col_name in cols:
    
    data.append(go.Scatter(x=tweet_ct['created_at_date'].dt.strftime(date_format='%Y-%m-%d'), y=tweet_ct[col_name],
                    #mode='lines',
                    name=col_name))
    #pdb.set_trace()
layout = go.Layout(dict(title = "Tweets Counts",
                  xaxis = dict(title = ''),
                  yaxis = dict(title = 'Tweet Count'),
                  ),legend=dict(
                orientation="h"))

py.iplot(dict(data=data, layout=layout), filename='basic-line', image_width=1200,
    image_height=800)

In [ ]:
soybean_tweet_lag = pd.read_csv('./Data_Graduate/soybeans_tweets_realDonaldTrump.csv')
print(f'{soybean_tweet_lag.shape[0]} samples and {soybean_tweet_lag.shape[1]} features in the Soybean Tweets dataset.')
soybean_tweet_lag['created_at']  = pd.to_datetime(soybean_tweet_lag['created_at'], format='%m-%d-%Y %H:%M:%S')
soybean_tweet_lag['created_at_date'] = pd.to_datetime([d.date() for d in soybean_tweet_lag['created_at']])
soybean_tweet_lag['created_at_time'] = [d.time() for d in soybean_tweet_lag['created_at']]
soybean_tweet_lag.head()

In [ ]:
soybean_countby_date = pd.DataFrame({'soybean_count' : soybean_tweet_lag.groupby( [ "created_at_date"] ).size()}).reset_index()
soybean_countby_date['created_at_date']  = pd.to_datetime(soybean_countby_date['created_at_date'], format='%Y-%m-%d')
#soybean_countby_date.head()

tweet_ct = soybean_countby_date #[soybean_countby_date["created_at_date"] >= "2019-04-22"]
tweet_ct.reset_index(inplace=True)
cols = list(tweet_ct)
unwanted_col = {"created_at_date", "index"} 
  
cols = [ele for ele in cols if ele not in unwanted_col] 

data = []
for col_name in cols:
    
    data.append(go.Scatter(x=tweet_ct['created_at_date'].dt.strftime(date_format='%Y-%m-%d'), y=tweet_ct[col_name],
                    #mode='lines',
                    name=col_name))
    #pdb.set_trace()
layout = go.Layout(dict(title = "Tweets soybean",
                  xaxis = dict(title = ''),
                  yaxis = dict(title = 'Tweet Count'),
                  ),legend=dict(
                orientation="h"))

py.iplot(dict(data=data, layout=layout), filename='basic-line', image_width=1200,
    image_height=800)

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from textblob import TextBlob
import string
import re
stop_words = set(stopwords.words('english'))

def _clean(txt): #test['name'] = df_ks['name'].apply(_clean)
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    txt = txt.lower()
    # punctuation removal 
    txt = ''.join(x for x in txt if x not in string.punctuation)
    txt = re.sub('[%s]' % re.escape(string.punctuation), ' ', txt)
    txt = re.sub('[‘’“”…]', ' ', txt)
    txt = re.sub('\n', ' ', txt)
    txt = re.sub('\w*\d\w*', ' ', txt)

    # stopwords removal  
    word_tokens = word_tokenize(txt)    
    #text_list = [w for w in word_tokens if not w in stop_words]  
    clean_txt = ""
  
    for w in word_tokens:
        if w.lower() not in stop_words:
            clean_txt += " "
            clean_txt += w 
    
    clean_txt = ' '.join(clean_txt.split()) # Removing multiple whitespaces
    noise = ['canceled']
    for ns in noise:
        clean_txt = clean_txt.replace(ns, "")

    return clean_txt

def syllable_count(word):
    word = word.lower()
    vowels = "aeiouy"
    count = 0
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
    if word.endswith("e"):
        count -= 1
    if count == 0:
        count += 1
    return count

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

def avg_word(sentence):
    words = sentence.split()
    return (sum(len(word) for word in words)/len(words))


def text_feat(df):
    # Function to calculate length of message excluding space
    df['text_len'] = df['text'].apply(lambda x: len(x) - x.count(" "))
    df['punct%'] = df['text'].apply(lambda x: count_punct(x))
    df["syllable_count"]   = df["text"].apply(lambda x: syllable_count(x))
    df["num_words"]  = df["text"].apply(lambda x: len(x.split()))
    df["num_chars"]  = df["text"].apply(lambda x: len(x.replace(" ","")))
    df['avg_word'] = df['text'].apply(lambda x: avg_word(x))
    df['num_stopwords'] = df['text'].apply(lambda x: len([x for x in x.split() if x in stop_words]))
    df['num_numerics'] = df['text'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
    df['num_capitalized'] = df['text'].apply(lambda x: len([x for x in x.split() if x.isupper()]))
    df['text'] = df['text'].apply(_clean)
    
    return df

print("Original Data Shape:", soybean_tweet_lag.shape)
soybean_tweet_lag_feat = text_feat(soybean_tweet_lag)
#df_text_feat_tfidf = name_tfidf(df_text_feat)
print("Added Text Features Shape:", soybean_tweet_lag_feat.shape)

In [ ]:
from wordcloud import WordCloud, STOPWORDS

# Thanks : https://www.kaggle.com/aashita/word-clouds-of-various-shapes ##
def plot_wordcloud(text, mask=None, max_words=200, max_font_size=100, title = None, title_size=40, image_color=False):
    stopwords = set(STOPWORDS)
    more_stopwords = {'school', 'miami', 'canceled'}
    stopwords = stopwords.union(more_stopwords)

    wordcloud = WordCloud(background_color='black',
                    stopwords = stopwords,
                    max_words = max_words,
                    max_font_size = max_font_size, 
                    random_state = 42,
                    width=800, 
                    height=400,
                    mask = mask)
    wordcloud.generate(str(text))
    
    #plt.figure(figsize=figure_size)
    if image_color:
        image_colors = ImageColorGenerator(mask);
        plt.imshow(wordcloud.recolor(color_func=image_colors), interpolation="bilinear");
        plt.title(title, fontdict={'size': title_size,  
                                  'verticalalignment': 'bottom'})
    else:
        plt.imshow(wordcloud);
        plt.title(title, fontdict={'size': title_size, 'color': 'black', 'verticalalignment': 'bottom'})
    plt.axis('off');
    plt.tight_layout()
    
    
plt.figure(figsize=(16,10))
#plt.suptitle('Bottom Performing Universities and Colleges (Some Campaign not ended)', fontsize=24)

#plt.subplot(2,2,1)
plot_wordcloud(soybean_tweet_lag_feat["text"], title="Twitter Text")

## Sentiment Exploration